**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [61]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense, Flatten
from keras.optimizers import sgd, adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Dropout

# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [62]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [63]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function ```act``` uses the learned policy, except that with probability ```epsilon```, it chooses a random action. Setting a non zero ```epsilon``` allows the agent to explore new policies.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [64]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat (rat?)
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256 # rat in black?
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        # forbidden borders in white?
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2) # in fact bernoulli
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [65]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=10 # set small when debugging
epochs_test=3 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The array ```position``` stores the information about the current state of the agent and the impossible positions at the borders. The array ```board``` stores the information about the presence of cheese or poison. Only part of the information in ```position``` and ```board``` will be released to the agent, only in the nearby cells.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [66]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.choice(4)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [67]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy: here we do not do anything
            #loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        
        ##### FILL IN HERE !!!!
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [68]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 12.0/18.0. Average score (-6.0)
Win/lose count 14.0/13.0. Average score (-2.5)
Win/lose count 7.0/12.0. Average score (-3.3333333333333335)
Final score: -3.3333333333333335


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




(1) The $Q$-function is defined as: $Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t=0}^\infty\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a]$. We get Bellman's equation by isolating the first term of the sum: $Q^\pi(s,a)=E_{p^{\pi}}[r(s, a) + \gamma \sum_{t=1}^\infty\gamma^{t-1}r(s_{t},a_{t})|s_{0}=s,a_{0}=a]$. The state and action at time $t=1$ are drawn according to $p^\pi(.|s, a)$ and we conclude by a change of variable in the sum that $Q^\pi(s,a)= r(s, a) + \gamma E_{(s^\prime, a^\prime)\sim p^\pi(.|s,a)}[Q^\pi(s^\prime, a^\prime)]$.

(2) The optimal $Q$-function is: $Q^*(s,a)=\max_{\pi}Q^\pi(s,a)$. Using the previous result, we can maximize it with respect to $\pi$. This is equivalent to maximizing the second term with respect to $a^\prime$, as it is the only variable controled by $\pi$, and we get: $Q^*(s,a)= r(s,a) + \gamma E_{(s^\prime, a^\prime)\sim p^{\pi^*}(.|s,a)} [\max_{a^\prime}Q^*(s^\prime,a^\prime)]$.

(3) We want to find the optimal $Q$-function. To achieve this, we must move the $Q$-function towards satisfying the optimal Bellman equation. Hence we minimize the distance between the two terms and hope that we will find the fixed point $Q^*$ that way.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [69]:
import random

class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory)>=self.max_memory:
            _ = self.memory.pop()
        self.memory.append(m)

    def random_access(self):
        return random.choice(self.memory)

***
The pipeline we will use for training is given below:

In [70]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterion.

*Another question you might ask is why did I say that the input of the neural net is simply a state? Doesn’t Q(s, a) take both a state and an action as an input? Well, there are two ways to approach this. One is to pass the action in as an input, and the other is to have one output per action (which is possible because we have relatively few actions). The latter option is much faster, as clearly explained by the DeepMind paper:*

*The main drawback of [passing the action as an input] is that a separate forward pass is required to compute the Q-value of each action, resulting in a cost that scales linearly with the number of actions. We instead use an architecture in which there is a separate output unit for each possible action, and only the state representation is an input to the neural network. The outputs correspond to the predicted Q-values of the individual action for the input state. The main advantage of this type of architecture is the ability to compute Q-values for all possible actions in a given state with only a single forward pass through the network.*

In [71]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state # each state is 5x5x2

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        # we only use V functions?
        return np.argmax(self.model.predict(s[None]))

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            ######## FILL IN
            s_, n_s_, a_, r_, game_over_ = self.memory.random_access()
            input_states[i] = s_
            if game_over_:
                Qf = np.zeros(4)
            else:
                Qf = self.model.predict(n_s_[None])[0]
            #print(Qf)
            target_q[i][a_] = r_ + self.discount * np.max(Qf) # size 4 ??
            #print(np.max(Qf))
                
        ######## FILL IN target q and input states
        # HINT: Clip the target to avoid exploding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        #print(input_states.shape, target_q.shape)
        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        ####### FILL IN
        model = Sequential()
        model.add(Dense(64, input_shape = (5, 5, 2), activation = 'relu'))
        model.add(Flatten())
        model.add(Dense(32, activation = 'relu'))
        model.add(Dense(4, activation = 'softmax'))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [72]:
epochs_train = 21

env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=500, batch_size = 32) #lr 0.01 mem 2000 bs 32
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train20.mp4'))

Epoch 000/021 | Loss 0.0320 | Win/lose count 4.0/6.0 (-2.0)
Epoch 001/021 | Loss 0.0175 | Win/lose count 4.5/3.0 (1.5)
Epoch 002/021 | Loss 0.0113 | Win/lose count 2.5/1.0 (1.5)
Epoch 003/021 | Loss 0.0207 | Win/lose count 1.5/2.0 (-0.5)
Epoch 004/021 | Loss 0.0127 | Win/lose count 0/1.0 (-1.0)
Epoch 005/021 | Loss 0.0184 | Win/lose count 2.0/9.0 (-7.0)
Epoch 006/021 | Loss 0.0203 | Win/lose count 1.5/5.0 (-3.5)
Epoch 007/021 | Loss 0.0710 | Win/lose count 2.0/1.0 (1.0)
Epoch 008/021 | Loss 0.0311 | Win/lose count 3.0/4.0 (-1.0)
Epoch 009/021 | Loss 0.0321 | Win/lose count 1.5/4.0 (-2.5)
Epoch 010/021 | Loss 0.0344 | Win/lose count 2.5/3.0 (-0.5)
Epoch 011/021 | Loss 0.0195 | Win/lose count 1.0/8.0 (-7.0)
Epoch 012/021 | Loss 0.0228 | Win/lose count 1.5/1.0 (0.5)
Epoch 013/021 | Loss 0.0358 | Win/lose count 1.5/7.0 (-5.5)
Epoch 014/021 | Loss 0.0340 | Win/lose count 2.0/4.0 (-2.0)
Epoch 015/021 | Loss 0.0232 | Win/lose count 1.5/3.0 (-1.5)
Epoch 016/021 | Loss 0.0432 | Win/lose count 2

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [73]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        
        model = Sequential()
        #model.add(Reshape((5, 5), input_shape = (5, 5, 2)))
        model.add(Conv2D(32, (3, 3), input_shape = (5, 5, 2), activation = 'relu')) # channels last
        model.add(Conv2D(16, (2, 2), activation = 'relu'))
        
        model.add(Flatten())
        model.add(Dense(32, activation = 'relu'))
        model.add(Dense(4, activation = 'softmax'))
        
        
        # Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization
        model.compile(sgd(lr=lr, momentum=0.0, decay=1e-4), "mse") # 
        self.model = model

In [74]:
epochs_train = 21
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=1e-1, epsilon = 0.1, memory_size=500, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train20.mp4'))

Epoch 000/021 | Loss 0.0228 | Win/lose count 0.5/2.0 (-1.5)
Epoch 001/021 | Loss 0.0385 | Win/lose count 2.0/5.0 (-3.0)
Epoch 002/021 | Loss 0.0280 | Win/lose count 0.5/4.0 (-3.5)
Epoch 003/021 | Loss 0.0312 | Win/lose count 3.0/4.0 (-1.0)
Epoch 004/021 | Loss 0.0241 | Win/lose count 1.0/6.0 (-5.0)
Epoch 005/021 | Loss 0.0310 | Win/lose count 1.0/4.0 (-3.0)
Epoch 006/021 | Loss 0.0141 | Win/lose count 3.5/7.0 (-3.5)
Epoch 007/021 | Loss 0.0533 | Win/lose count 4.5/2.0 (2.5)
Epoch 008/021 | Loss 0.0200 | Win/lose count 2.0/8.0 (-6.0)
Epoch 009/021 | Loss 0.0200 | Win/lose count 4.5/2.0 (2.5)
Epoch 010/021 | Loss 0.0348 | Win/lose count 1.5/2.0 (-0.5)
Epoch 011/021 | Loss 0.0388 | Win/lose count 2.0/3.0 (-1.0)
Epoch 012/021 | Loss 0.0284 | Win/lose count 2.0/2.0 (0.0)
Epoch 013/021 | Loss 0.0201 | Win/lose count 3.0/1.0 (2.0)
Epoch 014/021 | Loss 0.0318 | Win/lose count 5.0/3.0 (2.0)
Epoch 015/021 | Loss 0.0196 | Win/lose count 2.5/4.0 (-1.5)
Epoch 016/021 | Loss 0.0146 | Win/lose count 

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [75]:
epochs_test = 11

env = Environment(grid_size=size, max_time=T,temperature=0.3) # 0.1 0.3
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 1.5/4.0. Average score (-2.5)
Win/lose count 1.5/2.0. Average score (-1.5)
Win/lose count 2.5/4.0. Average score (-1.5)
Win/lose count 5.0/3.0. Average score (-0.625)
Win/lose count 1.0/1.0. Average score (-0.5)
Win/lose count 3.0/1.0. Average score (-0.08333333333333333)
Win/lose count 1.5/1.0. Average score (0.0)
Win/lose count 1.5/2.0. Average score (-0.0625)
Win/lose count 3.5/3.0. Average score (0.0)
Win/lose count 1.0/6.0. Average score (-0.5)
Win/lose count 2.0/5.0. Average score (-0.7272727272727273)
Final score: -0.7272727272727273
Test of the FC
Win/lose count 2.0/4.0. Average score (-2.0)
Win/lose count 0.5/5.0. Average score (-3.25)
Win/lose count 2.5/4.0. Average score (-2.6666666666666665)
Win/lose count 3.0/2.0. Average score (-1.75)
Win/lose count 2.5/3.0. Average score (-1.5)
Win/lose count 3.5/3.0. Average score (-1.1666666666666667)
Win/lose count 3.0/6.0. Average score (-1.4285714285714286)
Win/lose count 3.0/2.0. Average score (-1.125

In [76]:
HTML(display_videos('cnn_test10.mp4'))

In [77]:
HTML(display_videos('fc_test10.mp4'))

Usually, the agents tend to avoid the blue squares, but they do not explore the board. Instead, they keep a conservative policy that makes loops around itself. The convnet almost always achieves better results than the fully connected network.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [90]:
def train_explore(agent,env,epoch,prefix=''):
    
    ## use those samples of code:
    #In train explore:
    #state, reward, game_over = env.act(action, train=True)
    
    
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        
        agent.set_epsilon(agent.epsilon * 0.95) # decrease the parameter epsilon at each epoch

        while not game_over:
            # The agent performs an action (epsilon greedy)
            action = agent.act(state, train=True)
            
            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train = True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
    
    
    
    
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size)) # another board to store the malus

        # coordinate of the cat (rat?)
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256 # rat in black?
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        # forbidden borders in white?
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train = False):
        """This function returns the new state, reward and decides if the
        game ends."""
        
        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        
        #reward = self.board[self.x, self.y]
        # this line is replaced by the following block
        reward = 0
        if train: # dont apply it if in test
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.3 #0.1
        reward = reward + self.board[self.x, self.y]

        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        
        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2) # in fact bernoulli
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state
    
    

## In Environment exploring:
## You will have to change n_state to 3 because you will use one more layer! -> ok in the agent


In [91]:
# We change our CNN model to fit the right input

from keras import backend as K
keras.backend.set_learning_phase(1)

class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        model = Sequential()
        model.add(Conv2D(32, (3, 3), input_shape = (5, 5, 3), activation = 'relu')) # channels last
        model.add(Conv2D(16, (2, 2), activation = 'relu'))
        model.add(BatchNormalization())
        
        model.add(Flatten())
        model.add(Dense(32, activation = 'relu'))
        model.add(Dense(4, activation = 'softmax'))
        
        
        #model.add(BatchNormalization())
        #model.add(Dropout(.25))
        #model.add(AveragePooling2D())
        
        # Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization
        model.compile(sgd(lr=lr, momentum=0.0, decay=1e-4), "mse") # 
        self.model = model

In [94]:
# Training
epochs_train = 51
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=2., epsilon = 0.9, memory_size=500, batch_size = 32,n_state=3) # eps 0.9 lr 1.
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore50.mp4'))

Epoch 000/051 | Loss 0.0554 | Win/lose count 11.5/52.69999999999987 (-41.19999999999987)
Epoch 001/051 | Loss 0.0576 | Win/lose count 11.499999999999996/53.299999999999855 (-41.799999999999855)
Epoch 002/051 | Loss 0.0546 | Win/lose count 6.700000000000002/63.099999999999824 (-56.39999999999982)
Epoch 003/051 | Loss 0.0616 | Win/lose count 11.5/59.99999999999981 (-48.49999999999981)
Epoch 004/051 | Loss 0.0607 | Win/lose count 3.700000000000001/68.39999999999974 (-64.69999999999973)
Epoch 005/051 | Loss 0.0556 | Win/lose count 7.0000000000000036/64.79999999999978 (-57.799999999999784)
Epoch 006/051 | Loss 0.0546 | Win/lose count 3.900000000000001/68.29999999999971 (-64.39999999999971)
Epoch 007/051 | Loss 0.0448 | Win/lose count 3.800000000000001/71.99999999999973 (-68.19999999999973)
Epoch 008/051 | Loss 0.0514 | Win/lose count 5.000000000000002/62.49999999999977 (-57.49999999999977)
Epoch 009/051 | Loss 0.0750 | Win/lose count 7.800000000000004/57.59999999999985 (-49.79999999999985)


In [ ]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 5.0/6.0. Average score (-1.0)
Win/lose count 10.5/3.0. Average score (3.25)
Win/lose count 8.0/2.0. Average score (4.166666666666667)
Win/lose count 4.0/1.0. Average score (3.875)


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***